In [1]:
import numpy as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import math

In [39]:
class NaiveBayesClassifier:
    def __init__(self):
        self.classes = None
        self.means = None
        self.variances = None
        self.priors = None
        self.M_s=0
        self.Perror=0
        self.M = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features))   
        self.priors = np.zeros(n_classes)
    
        # 計算先驗機率
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = X_c.shape[0] / X.shape[0]
            if X_c.shape[0] == 0:
                self.means[i] = np.zeros(n_features)
                self.variances[i] = np.zeros(n_features)
            else:
                self.means[i] = np.mean(X_c, axis=0)
                self.variances[i] = np.var(X_c, axis=0) + 1e-9

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1) + 1

    def predict_proba(self, X):
        likelihood = np.zeros((X.shape[0], len(self.classes)))
        for i, c in enumerate(self.classes):
            class_likelihood = stats.norm.pdf(X, loc=self.means[i], scale=np.sqrt(self.variances[i]))
            class_likelihood = np.prod(class_likelihood, axis=1)
            likelihood[:, i] = class_likelihood * self.priors[i]
        evidence = np.sum(likelihood * self.priors, axis=1) + 1e-10
        return likelihood / evidence.reshape(-1, 1)

    def fit2(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features, n_features))   
        self.priors = np.zeros(n_classes)

        # 計算先驗機率
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = X_c.shape[0] / X.shape[0]
            if X_c.shape[0] == 0:
                self.means[i] = np.zeros(n_features)
                self.variances[i] = np.zeros((n_features, n_features))
            else:
                self.means[i] = np.mean(X_c, axis=0)
                self.variances[i] = np.cov(X_c, rowvar=False) + 1e-6*np.eye(n_features)
    

    def Bayes_error(self):
        # initialize M matrix
        n_classes = len(self.classes)
            
        if len(self.variances.shape) == 1:
            self.variances = self.variances.reshape(-1, 1)

        M = np.zeros((n_classes, n_classes))

        # calculate M matrix entries using loops
        for i in range(n_classes):
            for j in range(i+1, n_classes):
                delta = self.means[i]-self.means[j]
                sigma = 0.5*(self.variances[i]+self.variances[j])
                sigma_inv = np.linalg.pinv(sigma)
                m_ij = 0.125 * np.dot(delta.T, np.dot(sigma_inv, delta)) + 0.5 * np.log(np.linalg.det(sigma) / np.sqrt(np.linalg.det(self.variances[i]) * np.linalg.det(self.variances[j])))
                M[i,j] = m_ij
                M[j,i] = m_ij

        s_values = np.linspace(0.1, 0.9, 20)
        
        # 找出最佳的 s 值
        opt_s = 0
        min_error = float('inf')
        
        for s in s_values:    
            Perrors = []
            for i in range(3):
                for j in range(i+1, 3):
                    alpha_ij = math.sqrt(np.linalg.det(s*self.variances[i]+(1-s)*self.variances[j])/np.sqrt(np.linalg.det(self.variances[i])*np.linalg.det(self.variances[j])))
                    Perror = 0.25 * (1 - alpha_ij) * ((self.priors[i]*math.sqrt(s) - self.priors[j]*math.sqrt(1-s))**2 + (self.priors[i]*math.sqrt(1-s) - self.priors[j]*math.sqrt(s))**2)
                    Perrors.append(Perror)
            
            # 输出结果
            print(f"For s={s}:")
            for i in range(3):
                for j in range(i+1, 3):
                    print(f"Perror({i+1},{j+1}) = {Perrors[i*2+j-3]}")
            
            # 找到更好的 s 值時更新最佳 s 值與對應的誤差率
            min_idx = np.argmin(Perrors)
            if Perrors[min_idx] < min_error:
                min_error = Perrors[min_idx]
                opt_s = s_values[min_idx // 3]
                
        print("Optimal s: {}".format(opt_s))
        print("Bayes error rate: {}".format(min_error))


In [45]:
class Bayes:
    def __init__(self, training_data, labels):
        self.training_data = training_data
        self.labels = labels
        self.classes = np.unique(labels)
        self.class_num = len(self.classes)
        self.feature_num = training_data.shape[1]
        self.means = np.zeros([self.class_num, self.feature_num])
        self.variances = np.zeros([self.class_num, self.feature_num])
        self.priors = [0 for _ in range(self.class_num)]

    def fit(self):
        #calculate priors, means, variances
        for i, c in enumerate(self.classes):
            data_class = self.training_data[self.labels == c]
            self.priors[i] = len(data_class) / len(self.training_data)
            self.means[i] = np.mean(data_class, axis=0)
            self.variances[i] = np.var(data_class, axis=0) + 1e-10

    def predict(self, testing_data):
        likelihood = np.zeros([len(testing_data), self.class_num])

        for i, c in enumerate(self.classes):
            class_likelihood = stats.norm.pdf(testing_data, loc=self.means[i], scale=np.sqrt(self.variances[i]))
            class_likelihood = np.prod(class_likelihood, axis=1)
            likelihood[:, i] = class_likelihood * self.priors[i]
        return self.classes[np.argmax(likelihood, axis=1)]

    def score(self, testing_data, testing_labels):
        pre_labels = self.predict(testing_data)
        return np.sum([pre_labels[i] == testing_labels[i] for i in range(len(testing_data))]) / len(testing_data)

In [46]:
# 指定特徵名稱
feature_names = ['label',
                 'Alcohol', 
                 'Malic acid',
                 'Ash',
                 'Alcalinity of ash' ,
                 'Magnesium',
                 'Total phenols',
                 'Flavanoids',
                 'Nonflavanoid phenols',
                 'Proanthocyanins',
                 'Color intensity',
                 'Hue',
                 'OD280/OD315 of diluted wines',
                 'Proline' ]
data=pd.read_csv("wine.txt", names=feature_names)
data=data.sample(frac=1)

half = int(len(data) * 0.5)
train_data=data.iloc[:half]
test_data=data.iloc[half:]

train_labels = train_data.iloc[:,0].values
test_labels = test_data.iloc[:,0].values

train_data = train_data.drop('label', axis=1)
test_data = test_data.drop('label', axis=1)

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data.iloc[:, :])
test_data  = scaler.transform(test_data.iloc[:, :])

nb = NaiveBayesClassifier()
nb.fit(train_data, train_labels)
pred = nb.predict(test_data)
score = np.sum([pred[i] == test_labels[i] for i in range(len(test_data))]) / len(test_data)
score

0.9438202247191011

In [47]:
nb = NaiveBayesClassifier()
nb.fit(train_data, train_labels)
pred = nb.predict(test_data)
score_nb = np.sum([pred[i] == test_labels[i] for i in range(len(test_data))]) / len(test_data)

test = Bayes(train_data, train_labels)
test.fit()
pred = test.predict(test_data)
score_test = test.score(test_data, test_labels)

[[0.39722144 0.2712674  0.51017867 0.33763694 0.32831658 0.22213115
  0.11055647 0.15836656 0.51917201 0.23688232 0.24208257 0.29802727
  0.47191573]
 [0.50289638 1.16433263 1.39850484 1.03224218 1.60691685 0.86470027
  0.60248285 0.93180666 1.01955307 0.17351018 0.75222949 0.39279083
  0.28961617]
 [0.38411331 0.81690072 0.41517506 0.36962345 0.36062219 0.53877142
  0.12341206 1.15014068 0.73278312 1.08258178 0.2586773  0.13044894
  0.13301325]]
[[0.39722144 0.2712674  0.51017867 0.33763694 0.32831658 0.22213115
  0.11055647 0.15836656 0.51917201 0.23688232 0.24208257 0.29802727
  0.47191572]
 [0.50289638 1.16433263 1.39850484 1.03224218 1.60691685 0.86470027
  0.60248285 0.93180666 1.01955307 0.17351018 0.75222949 0.39279083
  0.28961616]
 [0.38411331 0.81690072 0.41517506 0.36962345 0.36062219 0.53877141
  0.12341206 1.15014068 0.73278312 1.08258178 0.2586773  0.13044894
  0.13301325]]
0.9438202247191011 0.9438202247191011
